In this tutorial, we'll use a transformer model to generate text based on input text. This is commonly referred to as text generation, and models like GPT (Generative Pretrained Transformer) and T5 are great for this task. We’ll use the popular model t5-base, which is a general-purpose text-to-text transformer that can perform tasks like summarization, translation, and text generation.

# Import the necessary libraries:

In [ ]:
from transformers import pipeline


In [1]:
from transformers import pipeline

Create a text generation pipeline:
# We’ll initialize a text-to-text generation pipeline using the t5-base model.

In [ ]:
generator = pipeline("text2text-generation", model="t5-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
gent=pipeline("text2text-generation", model="t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Provide input text:
You can provide some input text to generate a response. For this tutorial, we’ll use the T5 model in its basic form. You can experiment with tasks like answering questions or summarizing content.

In [ ]:
input_text = "Translate the following English text to French: Hugging Face is creating amazing tools for the NLP community."


In [3]:
text="transleting by using deep learning and ai models , is amazing "

# Generate the output:
Now you can use the generator() function to generate text based on the input.

In [ ]:
output = generator(input_text)
print("Generated text:", output[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated text: Hugging Face crée des outils extraordinaires pour la communauté de la LNP.


In [4]:
out=gent(text)
print("Generated text:", out[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated text: deep learning and ai models , is amazing . is amazing .



# Fine-Tuning the T5 Model for Text-to-Text Generation

In this section, we'll demonstrate how to fine-tune the `t5-base` model for text-to-text generation tasks on a custom dataset. Fine-tuning allows the model to adapt to specific text generation tasks such as translation, summarization, or other natural language processing tasks.

We will use the Hugging Face `Trainer` API to handle the fine-tuning process.


In [ ]:

# Step 1: Load a custom text-to-text dataset (e.g., a summarization dataset)
# For demonstration purposes, we’ll use the CNN/DailyMail dataset for summarization. You can replace this with your own dataset.

from datasets import load_dataset

dataset = load_dataset('cnn_dailymail', '3.0.0', split='train[:1%]')

# Step 2: Load the pretrained model and tokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Step 3: Preprocess the dataset for fine-tuning
def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]  # Prepend task for T5
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Tokenize the summaries (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_data = dataset.map(preprocess_data, batched=True, remove_columns=["article", "highlights"])

# Step 4: Define training arguments and initialize Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
evaluation_strategy="no",  # Disable evaluation
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data
)

# Step 5: Fine-tune the model
trainer.train()


In [2]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments


dataset = load_dataset('cnn_dailymail', '3.0.0', split='train[:1%]')



tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]  # Prepend task for T5
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding='max_length')
    model_inputs["labels"]=labels["input_ids"]
    return model_inputs

train_data=dataset.map(preprocess_data,batched=True,remove_columns=["article","highlights"])

traing=TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1
)

trainer=Trainer(
    model=model,
    args=traing,
    train_dataset=train_data
)

trainer.train()


Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: 

In [10]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


# Now check model's performance after fine-tuning

# Exercise Questions:






*   Experiment with different tasks, such as summarization or translation to various languages. How does the model perform when given more complex tasks (e.g., summarizing large paragraphs)?
*   Modify the script to allow the user to continuously input new tasks and generate text without having to restart the program.
*   Try out different models from Hugging Face’s text2text-generation models (e.g., t5-large, t5-small) to see how they compare in terms of performance and speed.
Explore fine-tuning the max_length and min_length parameters in the generation function to control the length of the output text.





In [3]:
from transformers import pipeline

# Define a function to load the model and tokenizer
def load_model(model_name):
    return pipeline("text2text-generation", model=model_name)

# Dictionary to store different models for easy switching
models = {
    "t5-small": load_model("t5-small"),
    "t5-large": load_model("t5-large"),
    # You can add more models from Hugging Face
}

# Function to process tasks based on user input
def process_task(model, task_type, text, max_length, min_length):
    if task_type == "summarization":
        return model(text, max_length=max_length, min_length=min_length)
    elif task_type == "translation":
        return model(text, max_length=max_length, min_length=min_length)
    else:
        return "Invalid task type. Use 'summarization' or 'translation'."

def main():
    # Infinite loop to continuously get user input
    while True:
        # Ask for the task type (e.g., summarization, translation)
        task_type = input("Enter task type (summarization/translation): ").strip()

        # Ask for the text input to summarize/translate
        text = input("Enter the text: ").strip()

        # Ask for the model name (e.g., t5-small, t5-large)
        model_name = input("Choose model (t5-small/t5-large): ").strip()

        if model_name not in models:
            print(f"Model {model_name} not available. Please choose a valid model.")
            continue

        # Get max_length and min_length for output control
        max_length = int(input("Enter max length for output: ").strip())
        min_length = int(input("Enter min length for output: ").strip())

        # Generate the output
        result = process_task(models[model_name], task_type, text, max_length, min_length)

        # Print the result
        print(f"\nGenerated {task_type} output: {result}\n")

if __name__ == "__main__":
    main()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Enter task type (summarization/translation): hello my name joud , am studing at sdaia bootcamp , i want to tell a story about a cat she was eating alot , tell she got fat then she decied to lose that fat so she stop eating , so she lost fat but after death .
Enter the text: i want to tell a story about a cat she was eating alot , tell she got fat then she decied to lose that fat so she stop eating , so she lost fat but after death .
Choose model (t5-small/t5-large): t5-large
Enter max length for output: 50
Enter min length for output: 20

Generated hello my name joud , am studing at sdaia bootcamp , i want to tell a story about a cat she was eating alot , tell she got fat then she decied to lose that fat so she stop eating , so she lost fat but after death . output: Invalid task type. Use 'summarization' or 'translation'.

Enter task type (summarization/translation): translation
Enter the text: hello my name joud , am studing at sdaia bootcamp , i want to tell a story about a cat she w

KeyboardInterrupt: Interrupted by user

In [ ]:
def main():
    # Infinite loop to continuously get user input
    while True:
        # Ask for the task type (e.g., summarization, translation)
        task_type = input("Enter task type (summarization/translation): ").strip()

        # Ask for the text input to summarize/translate
        text = input("Enter the text: ").strip()

        # Ask for the model name (e.g., t5-small, t5-large)
        model_name = input("Choose model (t5-small/t5-large): ").strip()

        if model_name not in models:
            print(f"Model {model_name} not available. Please choose a valid model.")
            continue

        # Get max_length and min_length for output control
        max_length = int(input("Enter max length for output: ").strip())
        min_length = int(input("Enter min length for output: ").strip())

        # Generate the output
        result = process_task(models[model_name], task_type, text, max_length, min_length)

        # Print the result
        print(f"\nGenerated {task_type} output: {result}\n")

if __name__ == "__main__":
    main()